#### 모듈 임포트

In [ ]:
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [ ]:
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
service = ChromeService(executable_path=ChromeDriverManager().install())

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 8.90MB/s]


In [ ]:
driver = webdriver.Chrome(service=service)

In [ ]:
driver.implicitly_wait(3)

### '서울정보소통광장' 크롤링

In [ ]:
## 서울정보소통광장 검색기간 기간설정 url
url = 'https://opengov.seoul.go.kr/sanction/list?rangeDate=custom'
driver.get(url)

In [ ]:
# 검색어 카테고리 -> '제목'
Category = driver.find_element(By.ID, 'searchCategory')
Category.send_keys('제목')

In [ ]:
# 검색어 입력
keyword = driver.find_element(By.ID, 'searchKeyword')
keyword.send_keys('민원 전동킥보드 무단방치 관련')

In [ ]:
# 검색기간 시작 날짜
startdate = '2019-07-01'
startDate = driver.find_element(By.ID,'startDate')
startDate.send_keys(startdate)

In [ ]:
# 검색기간 끝 날짜
enddate = '2021-07-14'
endDate = driver.find_element(By.ID,'endDate')
endDate.send_keys(enddate)

In [ ]:
# '검색' 버튼 클릭
xpath = '//*[@id="content"]/div/div[1]/div[1]/form/div[1]/div[2]/button[2]'
driver.find_element(By.XPATH, xpath).click()

In [ ]:
# html 파싱
page_source = driver.page_source 
soup = BeautifulSoup(page_source, "html.parser")

In [ ]:
# 검색결과 건수
cnt = soup.find('div', {'class':'al-left'}).text
cnt

'\n검색결과 : 260건\n\t\t\t\t\t\t\t'

In [ ]:
cnt = cnt.split(' ')[2].split('건')[0]
cnt

'260'

#### 건수 추출 함수로 정의

In [ ]:
import time

def search_cnt(keyword, startdate, enddate):
    url = 'https://opengov.seoul.go.kr/sanction/list?rangeDate=custom'
    driver.get(url)
    Category = driver.find_element(By.ID, 'searchCategory')
    Category.send_keys('제목')
    Keyword = driver.find_element(By.ID, 'searchKeyword')
    Keyword.send_keys(keyword)
    startDate = driver.find_element(By.ID,'startDate')
    startDate.send_keys(startdate)
    endDate = driver.find_element(By.ID,'endDate')
    endDate.send_keys(enddate)
    time.sleep(2)
    
    # 조회 버튼 클릭
    xpath = '//*[@id="content"]/div/div[1]/div[1]/form/div[1]/div[2]/button[2]'
    driver.find_element(By.XPATH, xpath).click()
    time.sleep(2)
    
    page_source = driver.page_source 
    soup = BeautifulSoup(page_source, "html.parser")
    
    cnt = soup.find('div', {'class':'al-left'}).text
    cnt = cnt.split(' ')[2].split('건')[0]
    return cnt

#### '민원 전동킥보드 무단방치 관련' 검색

In [ ]:
min_before = search_cnt('민원 전동킥보드 무단방치 관련', '2019-07-01', '2021-07-14')
min_before

'253'

In [ ]:
min_after = search_cnt('민원 전동킥보드 무단방치 관련', '2021-07-15', '2023-02-28')
min_after

'22'

#### '국민신문고 전동킥보드 무단방치 관련' 검색

In [ ]:
guk_before = search_cnt('국민신문고 전동킥보드 무단방치 관련', '2019-07-01', '2021-07-14')
guk_before

'16'

In [ ]:
guk_after = search_cnt('국민신문고 전동킥보드 무단방치 관련', '2021-07-15', '2023-02-28')
guk_after

'56'

#### 정책시행 이전, 이후 비교

In [ ]:
before = int(min_before) + int(guk_before)
after = int(min_after) + int(guk_after)

print(f'정책시행 이전(2019-07-01~2021-07-14) 민원 건수는 {before}, 정책시행 이후(2021-07-15~2023-02-28) 민원 건수는 {after}이다')

정책시행 이전(2019-07-01~2021-07-14) 민원 건수는 269, 정책시행 이후(2021-07-15~2023-02-28) 민원 건수는 78이다


In [ ]:
print(f'정책시행 이후 민원 감소율은 {(after-before)/before*100 :.2f}%이다.')

정책시행 이후 민원 감소율은 -71.00%이다.


### 월별 민원 건수 추출

#### 월별 시작 및 끝 날짜 생성

In [ ]:
monthStart = pd.date_range(start='2019-07-01', end='2023-02-28', freq='MS')
monthStart =  monthStart.strftime("%Y-%m-%d")
monthStart

Index(['2019-07-01', '2019-08-01', '2019-09-01', '2019-10-01', '2019-11-01',
       '2019-12-01', '2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
       '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01', '2020-09-01',
       '2020-10-01', '2020-11-01', '2020-12-01', '2021-01-01', '2021-02-01',
       '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01',
       '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01',
       '2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01',
       '2022-06-01', '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01',
       '2022-11-01', '2022-12-01', '2023-01-01', '2023-02-01'],
      dtype='object')

In [ ]:
monthEnd = pd.date_range(start='2019-07-01', end='2023-02-28', freq='BM')
monthEnd =  monthEnd.strftime("%Y-%m-%d")
monthEnd

Index(['2019-07-31', '2019-08-30', '2019-09-30', '2019-10-31', '2019-11-29',
       '2019-12-31', '2020-01-31', '2020-02-28', '2020-03-31', '2020-04-30',
       '2020-05-29', '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30',
       '2020-10-30', '2020-11-30', '2020-12-31', '2021-01-29', '2021-02-26',
       '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-30',
       '2021-08-31', '2021-09-30', '2021-10-29', '2021-11-30', '2021-12-31',
       '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-29', '2022-05-31',
       '2022-06-30', '2022-07-29', '2022-08-31', '2022-09-30', '2022-10-31',
       '2022-11-30', '2022-12-30', '2023-01-31', '2023-02-28'],
      dtype='object')

#### 월별 민원 건수 데이터프레임 생성

In [ ]:
Count = pd.DataFrame([], columns = ['시작날짜', '종료날짜', '"민원"건수', '"국민신문고"건수'])
for i in range(len(monthStart)):
    mcnt = search_cnt('민원 전동킥보드 무단방치 관련', monthStart[i], monthEnd[i])
    kcnt = search_cnt('국민신문고 전동킥보드 무단방치 관련', monthStart[i], monthEnd[i])
    Count.loc[i] = {'시작날짜': monthStart[i], '종료날짜': monthEnd[i],
                    '"민원"건수':mcnt, '"국민신문고"건수':kcnt}
Count

,시작날짜,종료날짜,"""민원""건수","""국민신문고""건수"
0,2019-07-01,2019-07-31,0,0
1,2019-08-01,2019-08-30,0,0
2,2019-09-01,2019-09-30,0,0
3,2019-10-01,2019-10-31,0,0
4,2019-11-01,2019-11-29,0,0
5,2019-12-01,2019-12-31,0,0
6,2020-01-01,2020-01-31,0,0
7,2020-02-01,2020-02-28,0,0
8,2020-03-01,2020-03-31,0,0
9,2020-04-01,2020-04-30,0,0


#### '민원'건수와 '국민신문고'건수 합치기

In [ ]:
Count['"민원"건수'] = Count['"민원"건수'].astype(int)
Count['"국민신문고"건수'] = Count['"국민신문고"건수'].astype(int)
Count.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 0 to 43
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   시작날짜       44 non-null     object
 1   종료날짜       44 non-null     object
 2   "민원"건수     44 non-null     int32 
 3   "국민신문고"건수  44 non-null     int32 
 4   건수         0 non-null      object
dtypes: int32(2), object(3)
memory usage: 1.7+ KB


#### csv 파일로 내보내기

In [ ]:
Count.to_csv('monthly_민원.csv')